<a href="https://colab.research.google.com/github/JuanOspina3/Proyecto-UDEA-ai4eng-20251---Pruebas-Saber-Pro-Colombia/blob/main/04%20-%20random_forest_ordinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 Importación de librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 📂 Carga de datos
train = pd.read_csv(r"C:\Users\juan2\OneDrive\Escritorio\train.csv")
test = pd.read_csv(r"C:\Users\juan2\OneDrive\Escritorio\test.csv")

X = train.drop(columns=["RENDIMIENTO_GLOBAL"])
y = train["RENDIMIENTO_GLOBAL"]
test_ids = test["ID"]

# 🔍 Columnas categóricas
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

# ⚙️ Preprocesamiento (solo categóricas)
preprocessor = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
    ]), cat_cols)
], remainder="passthrough")

# 🔗 Pipeline del modelo con Random Forest
model = Pipeline([
    ("pre", preprocessor),
    ("clf", RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        random_state=42,
        n_jobs=-1
    ))
])

# 🧪 División de datos
X_train, X_val, y_train, y_val = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 🚀 Entrenamiento y evaluación
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print(f"\n📊 Accuracy en validación (Random Forest): {acc:.5f}")

# 🏁 Entrenamiento final sobre todo el dataset
model.fit(X, y)
y_test_pred = model.predict(test)

# 💾 Exportar predicciones a CSV para Kaggle
submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": y_test_pred
})
submission.to_csv(r"C:\Users\juan2\OneDrive\Escritorio\submission_rf.csv", index=False)
print("\n✅ Archivo submission_rf.csv guardado correctamente.")
